In [1]:
%load_ext autoreload
%autoreload 2

from typing import Dict, List
import logging
import tempfile
import shutil
from pathlib import Path
from pprint import pprint
from models.utils.wandb import RunData
from models.utils.allennlp import (
    load_config, load_dataset_reader, load_iterator, load_model, load_modules, predict_loop_or_load, 
    load_outputs, create_onepass_generator)
from allennlp.models import Model
from allennlp.data import DatasetReader, DataIterator
import pandas as pd
from copy import deepcopy
import plotly.graph_objs as go
import numpy as np
from ipywidgets import interact, fixed
logging.basicConfig(
    format="%(asctime)s - %(levelname)s - %(name)s - %(message)s", level=logging.INFO)

logger = logging.getLogger(__name__)


In [2]:
# config
RUN = '9hfhh732'
EXTRA_MODULES = ['models', 'datasets']
DATA_DIR = '/Users/dhruv/UnsyncedDocuments/IESL/kb_completion/models/.data'
DOWNLOADS_DIR = '.models'
#DOWNLOADS_DIR = '/var/folders/r2/8mbb22091rb4vq4xtd19bbw80000gn/T/tmp5ypyp_4a'

In [3]:
# get the run files
logger.info("Downloading run files if needed...")
rd = RunData(RUN, group='iesl-boxes', project='kb-completion',
             download_dir=Path(DOWNLOADS_DIR))

rd.download_files()

2019-12-12 19:58:21,281 - INFO - __main__ - Downloading run files if needed...
2019-12-12 19:58:21,855 - INFO - models.utils.wandb - Setting up download dir as .models/mi2cxvo3
2019-12-12 19:58:22,062 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/best.th ()>
2019-12-12 19:58:24,548 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/config.json (application/json)>
2019-12-12 19:58:24,926 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics.json (application/json)>
2019-12-12 19:58:25,334 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_0.json (application/json)>
2019-12-12 19:58:25,838 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_1.json (application/json)>
2019-12-12 19:58:26,515 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_10.json (application/json)>
2019-12-12 19:58:27,076 - IN

2019-12-12 19:58:52,837 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_59.json (application/json)>
2019-12-12 19:58:53,365 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_6.json (application/json)>
2019-12-12 19:58:53,800 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_60.json (application/json)>
2019-12-12 19:58:54,316 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_61.json (application/json)>
2019-12-12 19:58:54,742 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_62.json (application/json)>
2019-12-12 19:58:55,180 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_63.json (application/json)>
2019-12-12 19:58:55,622 - INFO - models.utils.wandb - Downloading <File allennlp_serialization_dir/metrics_epoch_64.json (application/json)>
2019-12-12 19:

In [5]:

# load modules
logger.info("Loading {} for AllenNLP".format(EXTRA_MODULES))
load_modules(EXTRA_MODULES)

# override the val dataset setup to become the test dataset setup
overrides = {'validation_dataset_reader':
             {
                 "file_reader": {
                     "type": "rank-val-id-reader"
                 },
                 'all_datadir': DATA_DIR,
                 'dataset_name':"WN18RR_HYP_REV"
             },
             'dataset_reader': {
                 'all_datadir': DATA_DIR,
                 'dataset_name': "WN18RR_HYP_REV"
             }
             }

logger.info("Setting up config overrides ...")

# load config
config = load_config(
    rd.download_dir/"allennlp_serialization_dir", overrides_dict=overrides)
#pprint(config.as_dict())

# load model

model = load_model(rd.download_dir/"allennlp_serialization_dir", config=config)

# create dataset reader
dataset_reader = load_dataset_reader(config=config)

# create iterator
# This will take time for the first time as it will create the cache
iterator = load_iterator(config=config)
generator = create_onepass_generator(iterator, dataset_reader)

2019-12-12 20:04:11,788 - INFO - __main__ - Loading ['models', 'datasets'] for AllenNLP
2019-12-12 20:04:11,864 - INFO - __main__ - Setting up config overrides ...
2019-12-12 20:04:11,899 - INFO - allennlp.common.params - Converting Params object to dict; logging of default values will not occur when dictionary parameters are used subsequently.
2019-12-12 20:04:11,900 - INFO - allennlp.common.params - CURRENTLY DEFINED PARAMETERS: 
2019-12-12 20:04:11,900 - INFO - allennlp.common.params - datasets_for_vocab_creation = []
2019-12-12 20:04:11,901 - INFO - allennlp.common.params - validation_data_path = dummy_path
2019-12-12 20:04:11,902 - INFO - allennlp.common.params - train_data_path = dummpy_path
2019-12-12 20:04:11,903 - INFO - allennlp.common.params - dataset_reader.all_datadir = /mnt/nfs/work1/mccallum/dhruveshpate/kb_completion/models/.data
2019-12-12 20:04:11,904 - INFO - allennlp.common.params - dataset_reader.dataset_name = WN18RR_HYP_REV
2019-12-12 20:04:11,905 - INFO - allenn

2019-12-12 20:04:12,207 - INFO - allennlp.common.from_params - instantiating class <class 'datasets.file_readers.file_reader.FileReader'> from params {'type': 'rank-val-id-reader'} and extras {'dataset_dir'}
2019-12-12 20:04:12,211 - INFO - allennlp.common.params - validation_dataset_reader.file_reader.type = rank-val-id-reader
2019-12-12 20:04:12,216 - INFO - allennlp.common.from_params - instantiating class <class 'datasets.file_readers.openke.RankValIdReader'> from params {} and extras {'dataset_dir'}
2019-12-12 20:04:12,217 - INFO - allennlp.common.from_params - instantiating class <class 'allennlp.data.iterators.data_iterator.DataIterator'> from params {'batch_size': 1, 'cache_instances': True, 'type': 'single-sample-rank-validation-iterator'} and extras set()
2019-12-12 20:04:12,222 - INFO - allennlp.common.params - validation_iterator.type = single-sample-rank-validation-iterator
2019-12-12 20:04:12,225 - INFO - allennlp.common.from_params - instantiating class <class 'datasets.

In [6]:
# Predict
results = predict_loop_or_load(model, generator, device='cpu', output_file=Path(rd.download_dir)/'test_outputs_VAL_TC.jsonl')

2019-12-12 20:07:04,302 - INFO - models.utils.allennlp - Starting predictions ...
1it [00:00,  1.67it/s]

0  :  {"h": 4410, "t": 33512, "r": 0, "hr_rank": 1492, "tr_rank": 506, "avg_rank": 999.0, "hr_mrr": 0.0006697923643670462, "tr_mrr": 0.0019723865877712033, "int_vol": -172.6737060546875, "loss": null}



2it [00:01,  1.84it/s]

1  :  {"h": 16031, "t": 32320, "r": 1, "hr_rank": 11332, "tr_rank": 40188, "avg_rank": 25760.0, "hr_mrr": 8.823788934968675e-05, "tr_mrr": 2.4882430515812785e-05, "int_vol": -307.2060546875, "loss": null}



3it [00:01,  1.97it/s]

...

3033it [21:32,  2.28it/s]

3032  :  {"h": 32242, "t": 9602, "r": 0, "hr_rank": 14453, "tr_rank": 25949, "avg_rank": 20201.0, "hr_mrr": 6.918500069185e-05, "tr_mrr": 3.853564547206166e-05, "int_vol": -234.12551879882812, "loss": null}



3034it [21:32,  2.26it/s]

3033  :  {"h": 785, "t": 11138, "r": 8, "hr_rank": 518, "tr_rank": 2256, "avg_rank": 1387.0, "hr_mrr": 0.0019267822736030828, "tr_mrr": 0.00044306601683650863, "int_vol": -146.9150390625, "loss": null}



3034it [21:33,  2.35it/s]


## Analysis

In [7]:
# Some plotting functions
def get_x_for_hist(df: pd.DataFrame, filter_on = None, filter_value = 0, plot_ = 'avg_rank'):
    if filter_on is None:
        return df[plot_]
    x = df[df[filter_on] == filter_value][plot_]
    return x

def create_hist_widget(df: pd.DataFrame, filter_on = None, filter_value = 0, plot_ = 'avg_rank'):
    widget = go.FigureWidget()
    x = get_x_for_hist(df, filter_on, filter_value=filter_value, plot_=plot_)
    widget.add_histogram(x=x, xbins=dict( # bins used for histogram
        size=5, end=5000
    ))
    return widget

def update_hist_widget(wid: go.FigureWidget, df: pd.DataFrame, filter_on = None, filter_value = 0, plot_ = 'avg_rank', bin_size=10, end=2000):
    x = get_x_for_hist(df, filter_on=filter_on, filter_value=filter_value, plot_=plot_)
    wid.data[0].x=x
    wid.data[0].xbins.size = bin_size
    wid.data[0].xbins.end = end
    return wid


In [8]:
results_df = pd.DataFrame(results)
results_df

,avg_rank,h,hr_mrr,hr_rank,int_vol,loss,r,t,tr_mrr,tr_rank
0,999.0,4410,0.000670,1492,-172.673706,None,0,33512,0.001972,506
1,25760.0,16031,0.000088,11332,-307.206055,None,1,32320,0.000025,40188
2,5839.5,40543,0.000147,6795,-180.566589,None,5,172,0.000205,4884
3,16007.0,15278,0.022222,44,-140.964966,None,4,39692,0.000031,31970
4,2217.5,14838,0.014085,70,-165.351471,None,0,22936,0.000229,4365
5,3112.5,31887,0.000739,1353,-166.933929,None,2,743,0.000205,4872
6,15897.0,27360,0.001002,997,-203.350708,None,0,29405,0.000032,30797
7,2654.5,35494,0.500000,1,-100.862740,None,1,15192,0.000188,5308
8,390.5,18925,0.500000,1,-38.718956,None,1,32325,0.001280,780
9,17779.5,3750,0.083333,11,-122.663422,None,4,40324,0.000028,35548


In [9]:
interact(update_hist_widget, 
         wid=fixed(create_hist_widget(results_df)),
         df=fixed(results_df),
        filter_on = [None, 'r'],
        filter_value = list(range(11)),
        plot_=['hr_mrr', 'tr_mrr','avg_rank','hr_rank','tr_rank'],
        bin_size = list(np.linspace(0.01,1.0,num=10)) + list(range(1,22,2)),
        end=(1,10000, 1))

interactive(children=(Dropdown(description='filter_on', options=(None, 'r'), value=None), Dropdown(description…

<function __main__.update_hist_widget(wid: plotly.graph_objs._figurewidget.FigureWidget, df: pandas.core.frame.DataFrame, filter_on=None, filter_value=0, plot_='avg_rank', bin_size=10, end=2000)>

In [ ]:
results=load_outputs('.models/i1gq68qi/test_outputs_VAL_TC.jsonl')

In [ ]:
results_df = pd.DataFrame(results)
(results_df['avg_rank'] - 1).mean()

In [10]:
sum(results_df['hr_rank'] <10)/len(results_df) 

0.4024390243902439

In [11]:
sum(results_df['tr_rank'] <10)/len(results_df) 

0.005273566249176005

In [12]:
id2r = {0: "_hypernym",
        1: "_derivationally_related_form",
        2: "_instance_hypernym",
        3: "_also_see",
        4: "_member_meronym",
        5: "_synset_domain_topic_of",
        6: "_has_part",
        7: "_member_of_domain_usage",
        8: "_member_of_domain_region",
        9: "_verb_group",
        10: "_similar_to"
        }

In [14]:
fig = go.Figure()
for r in range(0,11):
    fig.add_trace(go.Histogram(
        x=get_x_for_hist(results_df, filter_on='r',filter_value=r, plot_='hr_mrr'),
        #histnorm='percent',
        name=id2r[r], # name used in legend and hover labels
        xbins=dict( # bins used for histogram
            end=1.0,
            size=0.05
        ),
        opacity=0.75
    ))

fig.layout.update(
    title={
        'text': 'WN18RR Average Rank accros relations',
        'y':0.9,
        'x':0.7,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title_text='MRR', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.1, # gap between bars of adjacent location coordinates
    bargroupgap=0.02, # gap between bars of the same location coordinates
    legend = go.layout.Legend(
        x=0.1,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=1
    )
)

fig.show()

In [ ]:
fig = go.Figure()
for r in range(0,11):
    fig.add_trace(go.Histogram(
        x=get_x_for_hist(results_df, filter_on='r',filter_value=r, plot_='avg_rank'),
        #histnorm='percent',
        name=id2r[r], # name used in legend and hover labels
        xbins=dict( # bins used for histogram
            start=0,
            end=500,
            size=50
        ),
        opacity=0.75
    ))

fig.layout.update(
    title={
        'text': 'WN18RR Average Rank accros relations',
        'y':0.9,
        'x':0.7,
        'xanchor': 'center',
        'yanchor': 'top'},
    xaxis_title_text='Rank', # xaxis label
    yaxis_title_text='Count', # yaxis label
    bargap=0.5, # gap between bars of adjacent location coordinates
    bargroupgap=0.02, # gap between bars of the same location coordinates
    legend = go.layout.Legend(
        x=0.1,
        y=1,
        traceorder="normal",
        font=dict(
            family="sans-serif",
            size=12,
            color="black"
        ),
        bgcolor="LightSteelBlue",
        bordercolor="Black",
        borderwidth=2
    ),
    margin=go.layout.Margin(
        l=0,
        r=0,
        b=0,
        t=0,
        pad=1
    )
)

fig.show()

In [ ]:
results_df['tr_mrr'].mean()

In [ ]:
results_df['hr_mrr'].mean()